分析:

1. 订单表的长度, shape, columns
2. 统计菜名的平均菜价(amounts)
3. 什么菜最受欢迎
4. 哪个订单ID点的菜最多

In [3]:
import pandas as pd 
import matplotlib.pyplot as plt

# 1. Load Data
df = pd.read_excel('data/meal_order_detail.xlsx')
df.info()

# 2. Data Preprocessing (NA)

# 3. Data Visualisation





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2779 entries, 0 to 2778
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   detail_id          2779 non-null   int64         
 1   order_id           2779 non-null   int64         
 2   dishes_id          2779 non-null   int64         
 3   logicprn_name      0 non-null      float64       
 4   parent_class_name  0 non-null      float64       
 5   dishes_name        2779 non-null   object        
 6   itemis_add         2779 non-null   int64         
 7   counts             2779 non-null   int64         
 8   amounts            2779 non-null   int64         
 9   cost               0 non-null      float64       
 10  place_order_time   2779 non-null   datetime64[ns]
 11  discount_amt       0 non-null      float64       
 12  discount_reason    0 non-null      float64       
 13  kick_back          0 non-null      float64       
 14  add_inpr